In [47]:
import pandas as pd
import os
import re
from indicnlp.tokenize import indic_tokenize

In [48]:
df = pd.read_csv(r"GeneratedDatasets/hindi_scrapped_processed.csv")

In [49]:
# count the number of non null values for each feature
df.count()

url           1000
content        996
word_count    1000
dtype: int64

In [50]:
dataframe = df.dropna()
print(dataframe.count())

url           996
content       996
word_count    996
dtype: int64


In [51]:
dataframe["content"][995] # To ensure that the text is cleaned

'पर बाद शामिल हुए देश। शक्तियाँयाऐलाइड शक्तियाँअंग्रेज़ी उन देशों का था जिन्होनेंद्वितीय विश्वयुद्धमेंब्रिटेनसंयुक्त संघऔरचीनका साथ दिया औरअक्ष शक्तियों के ख़िलाफ़ लड़े। द्वितीय की मुठभेड़ में इसलिए शामिल हुए क्योंकि उनपर अक्ष देशों कर दिया उन्हें अपने ऊपर होने का था फिर उन्हें थी कि अक्ष शक्तियाँ गयी पूरी पर जाएँगी। सितम्बर में की शुरआत मेंफ़्रांसपोलैंडऔरयूनाइटेड किंगडम में थे। जल्द ब्रिटेन के कुछ ज़ीलैंडऔरदक्षिण अफ़्रीकाभी इस में गए। के बाद का ब्रिटेन और मिलकर किया।भारतबेल्जियमयूनानमेक्सिकोचेकोस्लोवाकियानॉर्वेनेदरलैंड्ज़इथियोपियाऔरब्राज़ीलमें में थे। में जाकर शक्तियों की होने पर अक्ष शक्तियों का ख़त्म गया। इन्हें भी देखें मशीन सितंबर'

* **Challenge 1**: As we can see in the above text the words of the extracted text seemed to be concatenated (**Compound words)** which could significantly effect the training. 
* **Solution** : Split the concatenated text into meaningful words

In [52]:
text = dataframe["content"][995]
segmented_text = " ".join(indic_tokenize.trivial_tokenize(text, lang='hi'))
print(segmented_text) # lack of the proper hindi word dictionary

पर बाद शामिल हुए देश । शक्तियाँयाऐलाइड शक्तियाँअंग्रेज़ी उन देशों का था जिन्होनेंद्वितीय विश्वयुद्धमेंब्रिटेनसंयुक्त संघऔरचीनका साथ दिया औरअक्ष शक्तियों के ख़िलाफ़ लड़े । द्वितीय की मुठभेड़ में इसलिए शामिल हुए क्योंकि उनपर अक्ष देशों कर दिया उन्हें अपने ऊपर होने का था फिर उन्हें थी कि अक्ष शक्तियाँ गयी पूरी पर जाएँगी । सितम्बर में की शुरआत मेंफ़्रांसपोलैंडऔरयूनाइटेड किंगडम में थे । जल्द ब्रिटेन के कुछ ज़ीलैंडऔरदक्षिण अफ़्रीकाभी इस में गए । के बाद का ब्रिटेन और मिलकर किया । भारतबेल्जियमयूनानमेक्सिकोचेकोस्लोवाकियानॉर्वेनेदरलैंड्ज़इथियोपियाऔरब्राज़ीलमें में थे । में जाकर शक्तियों की होने पर अक्ष शक्तियों का ख़त्म गया । इन्हें भी देखें मशीन सितंबर


* **Limitation** : The existing libraries doesnt have the capability of splitting the **compound words** into meaningful words 
* **Solution** : We need a language aware approach like identifying the kind of concatenations and define a dictionaries of that categories and use the dictionary to split it. 

In [53]:
hindi_dict = {
    # States
    "आंध्र प्रदेश", "अरुणाचल प्रदेश", "असम", "बिहार", "छत्तीसगढ़",
    "गोवा", "गुजरात", "हरियाणा", "हिमाचल प्रदेश", "झारखंड",
    "कर्नाटक", "केरल", "मध्य प्रदेश", "महाराष्ट्र", "मणिपुर",
    "मेघालय", "मिजोरम", "नागालैंड", "ओडिशा", "पंजाब",
    "राजस्थान", "सिक्किम", "तमिल नाडु", "तेलंगाना", "त्रिपुरा",
    "उत्तर प्रदेश", "उत्तराखंड", "पश्चिम बंगाल",
    
    # Union Territories
    "अंडमान और निकोबार", "चंडीगढ़", "दादरा और नगर हवेली", "दमन और दीव",
    "दिल्ली", "जम्मू और कश्मीर", "लद्दाख", "लक्षद्वीप", "पुडुचेरी",

    # Countries
    "अफ़गानिस्तान", "अल्बानिया", "अल्जीरिया", "अर्जेंटीना", "ऑस्ट्रेलिया",
    "ऑस्ट्रिया", "बहरीन", "बांग्लादेश", "बेलारूस", "बेल्जियम",
    "ब्राज़ील", "कनाडा", "चीन", "मिस्र", "फ़्रांस",
    "जर्मनी", "यूनान", "हंगरी", "भारत", "इंडोनेशिया",
    "ईरान", "इराक", "आयरलैंड", "इज़राइल", "इटली",
    "जापान", "केन्या", "मलेशिया", "मेक्सिको", "नेपाल",
    "नीदरलैंड", "नॉर्वे", "पाकिस्तान", "पेरू", "फिलीपींस",
    "पोलैंड", "रूस", "सऊदी अरबिया", "दक्षिण अफ्रीका", "दक्षिण कोरिया",
    "स्पेन", "श्रीलंका", "स्वीडन", "स्विट्ज़रलैंड", "थाईलैंड",
    "तुर्की", "यूक्रेन", "संयुक्त अरब अमीरात", "यूनाइटेड किंगडम", "अमेरिका",
    "वियतनाम", "ज़िम्बाब्वे",
    
    # Additional Entries
    "भारत", "बेल्जियम", "यूनान", "मेक्सिको", "चेको", "स्लोवाकिया", "नॉर्वे", "ने",
    "दरलैंड्ज़", "इथियोपिया", "और", "ब्राज़ील", "में"
}

In [58]:
# Use the longest match first approach
def split_compound(text, hindi_dict):
    words = []
    text = text.strip()

    while text:
        matched = False

        for length in range(len(text), 0, -1):  # Start from the largest possible match
            part = text[:length]
            if part in hindi_dict:
                words.append(part)
                text = text[length:]  # Remove the matched part
                matched = True
                break  # Stop as soon as a match is found

        if not matched:
            # If no match is found, keep the remaining text as a single word
            words.append(text)
            break

    return str(" ".join(words))

input_text = "भारतबेल्जियमयूनानमेक्सिकोचेकोस्लोवाकियानॉर्वेनेदरलैंड्ज़इथियोपियाऔरब्राज़ीलमें"
split_words = split_compound(input_text, hindi_dict)
print(split_words)

भारत बेल्जियम यूनान मेक्सिको चेको स्लोवाकिया नॉर्वे ने दरलैंड्ज़ इथियोपिया और ब्राज़ील में


In [59]:
dataframe["segmented_content"] = dataframe["content"].apply(lambda text: split_compound(text, hindi_dict))

C:\Users\masab\AppData\Local\Temp\ipykernel_16088\3033079495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["segmented_content"] = dataframe["content"].apply(lambda text: split_compound(text, hindi_dict))


In [60]:
dataframe

,url,content,word_count,segmented_content
0,https://hi.wikipedia.org/wiki/%E0%A4%B9%E0%A5%...,हैदराबाद के निज़ामउलमुल्क। इस में किमी का क्षे...,500,हैदराबाद के निज़ामउलमुल्क। इस में किमी का क्षे...
1,https://hi.wikipedia.org/wiki/%E0%A4%A8%E0%A4%...,मीर क़मरउददीन ख़ान सिद्दिक़ीउर्फ़निजामउलमुल्क ...,384,मीर क़मरउददीन ख़ान सिद्दिक़ीउर्फ़निजामउलमुल्क ...
2,https://hi.wikipedia.org/wiki/%E0%A4%94%E0%A4%...,ओरंगाबाद भारत राज्यमें स्थित है। यह का भी है औ...,500,ओरंगाबाद भारत राज्यमें स्थित है। यह का भी है औ...
3,https://hi.wikipedia.org/wiki/%E0%A4%94%E0%A4%...,ओरंगाबाद भारत राज्यमें स्थित है। यह का भी है औ...,500,ओरंगाबाद भारत राज्यमें स्थित है। यह का भी है औ...
4,https://hi.wikipedia.org/wiki/%E0%A4%AC%E0%A5%...,बीबी के मुग़ल बादशाह औरंगज़ेब इन्होने अंतिम सत...,342,बीबी के मुग़ल बादशाह औरंगज़ेब इन्होने अंतिम सत...
...,...,...,...,...
995,https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%...,पर बाद शामिल हुए देश। शक्तियाँयाऐलाइड शक्तियाँ...,173,पर बाद शामिल हुए देश। शक्तियाँयाऐलाइड शक्तियाँ...
996,https://hi.wikipedia.org/wiki/%E0%A4%A8%E0%A4%...,मैंसीडोनिया मॉन्टेनीग्रो उत्तरी संधि संगठन सैन...,500,मैंसीडोनिया मॉन्टेनीग्रो उत्तरी संधि संगठन सैन...
997,https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%...,अल्बानिया दक्षिणपूर्वयूरोपमें स्थित है।इसकी भू...,500,अल्बानिया दक्षिणपूर्वयूरोपमें स्थित है।इसकी भ...
998,https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%...,अल्बानिया का ध्वजहै।,4,अल्बानिया का ध्वजहै।


In [61]:
training_datafile = r"Training/TrainingReadyHindiData.txt"

if not os.path.exists(training_datafile):
    with open(training_datafile, "w") as f:
        f.write("")

with open(training_datafile, "w", encoding='utf-8') as f:
    for col in dataframe["segmented_content"]:
        f.write(col + "\n")  # Write each column name on a new line